## Pipeline

In [1]:
from autointent import Context
from autointent.pipeline.optimization.utils import get_run_name, load_data


run_name = get_run_name("multiclass-cpu")

data = load_data("/home/voorhs/repos/AutoIntent/tests/assets/data/clinc_subset_multiclass.json")
context = Context(
    dataset=data,
    test_dataset=None,
    device="cpu",
    multilabel_generation_config="",
    db_dir="assets",
    regex_sampling=0,
    seed=0,
    dump_dir="assets"
)

In [2]:
nodes = [
    {
        'metric': 'retrieval_hit_rate_intersecting',
        'node_type': 'retrieval',
        'search_space': [
            {
                'k': [10],
                'model_name': ["sentence-transformers/all-MiniLM-L6-v2"],
                'module_type': 'vector_db'
            }
        ]
    },
    {
        'metric': 'scoring_roc_auc',
        'node_type': 'scoring',
        'search_space': [{'module_type': 'linear', "n_jobs": [1]}]
    },
    {
        'metric': 'prediction_accuracy',
        'node_type': 'prediction',
        'search_space': [{'module_type': 'tunable'}]
    }
]

In [3]:
from autointent.pipeline import PipelineOptimizer

pipeline = PipelineOptimizer.from_dict_config({'nodes': nodes})

In [4]:
pipeline.optimize(context)

[I 2024-10-22 12:51:21,643] A new study created in memory with name: no-name-6e4e33b0-2d04-4481-aab0-21be410bec4f


In [9]:
from pathlib import Path

pipeline.dump(Path.cwd() / "logs")